In [3]:
# Takes LAMOST spectra compares retrievals between rrlfe and from the catalog in 
# Table 6 of Liu+ 2020 ApJSS 247:68 'Probing the Galactic Halo with RR Lyrae Stars. I. The Catalog'.

# Created 2023 Jan. 5 by E.S.

In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

%matplotlib qt

In [5]:
stem = "/Users/bandari/Documents/git.repos/rrlfe/"

In [14]:
# read in Liu+ catalog

df_liu_stars = pd.read_csv(stem + "notebooks_for_development/spec_sets_check/lamost/catalog_liu_et_al.txt",
                              skiprows=65, delim_whitespace=True, usecols=[0,1,2,11,12,13,14,31], 
                               names=["ID","RAdeg","DEdeg","VType","SNR","FeH","e_FeH","Num"])

In [17]:
# read in RA, DEC info for the LAMOST stars

df_our_sdss_stars = pd.read_csv(stem + "notebooks_for_development/data/Skyserver_CrossID1_4_2023_5_21_47_AM.csv")

In [19]:
# give some cols same name, so we can merge on them

df_liu_stars["ra"] = df_liu_stars["RAdeg"]
df_liu_stars["dec"] = df_liu_stars["DEdeg"]

# some rounding is necessary to enable matching
df_liu_stars["ra_round"] = np.round(df_liu_stars["ra"],2)
df_our_sdss_stars["ra_round"] = np.round(df_our_sdss_stars["ra"],2)
df_liu_stars["dec_round"] = np.round(df_liu_stars["dec"],3)
df_our_sdss_stars["dec_round"] = np.round(df_our_sdss_stars["dec"],3)

In [20]:
# this merges tables based on RA, DEC, and leaves the Liu FeH in the final table
# (but our FeH is still missing)

merged_df_liu_feh = pd.merge(df_liu_stars, df_our_sdss_stars, how="inner", on=["ra_round","dec_round"])

In [21]:
# read in our retrieved Fe/H and splice stuff to enable matching

df_our_data = pd.read_csv(stem + " [ TABLE WITH RETRIEVED FE/H FROM LAMOST SPECTRA ]")

In [9]:
# STUFF BELOW SPECIFIC TO SDSS, BUT MAY YET BE USEFUL
split_1 = df_our_data["orig_spec_file_name"].str.split("-", 3, expand=True)
split_2 = split_1[3].str.split("_", 1, expand=True)

In [10]:
df_our_data["plate"] = split_1[1].astype(int)
df_our_data["mjd"] = split_1[2].astype(int)
df_our_data["fiberid"] = split_2[0].astype(int)

In [42]:
# our stars can have multiple FeHs, because of multiple epochs
# (an 'inner' merge in this context means that repeats are allowed, but any rows with NaNs 
# left in either FeH column is removed)

merged_df_whitten_and_our_feh = pd.merge(df_our_data, merged_df_whitten_feh, on=["plate", "mjd", "fiberid"], 
                                         how="inner")

In [45]:
plt.scatter(merged_df_whitten_and_our_feh["feh_retrieved"],merged_df_whitten_and_our_feh["NET_FEH"])
plt.plot([-4.0,4.0],[-4.0,4.0], linestyle="--")
plt.xlabel("Our FeH")
plt.ylabel("Whitten+ FeH")
plt.show()